### 一等对象的定义
- 在运行时创建
- 能赋值给变量
- 能作为参数传递给函数
- 能作为函数的返回结果

In [56]:
# 插一个map函数。map(function, iterable, ...)
def examp(arg):
    return arg*arg
x = map(examp, range(5))
print(x, list(x))

<map object at 0x000002BB06BB30B8> [0, 1, 4, 9, 16]


### 高阶函数
- 接收函数作为参数，或者把函数作为结果返回的函数是高阶函数。

### 函数式编程的特点之一就是使用高阶函数

In [57]:
# 如果想按照单词的长度排序，只需要将len函数作为参数传递给sorted函数
words = ["adf", "dafdsf", "dsfasdffa", "rdfadsf"]
c = sorted(words, key=len)
print(c)

# 只要是单参数的函数，都可以传递给key
def reverse(arg):
    return arg[::-1]
d = sorted(words, key=reverse)
print(d)

['adf', 'dafdsf', 'rdfadsf', 'dsfasdffa']
['dsfasdffa', 'adf', 'rdfadsf', 'dafdsf']


- 函数式语言通常会提供map，filter和reduce三个高阶函数。在python中，列表推导和生成器表达式具有map和filter两个函数的功能，且更易阅读。
- all(iterable) 全为真返回真
- any(iterable) 只要有真就返回真

In [58]:
a = all([0,1,1,1])
b = any([0,1,1])
print(a,b)

False True


### 匿名函数 lambda
- 在参数列表中最适合用匿名函数
- 除此之外，很少用匿名函数

### 可调用对象
- 定义就是能加()运行
- 判断能否可调用，用callable函数

In [14]:
a = 12
callable(a)

False

In [59]:
# 用户定义把类的实例变成可调用对象
# 其实任何python对象都可以表现的像函数，只需要实现__call__方法
print.__call__(1)

import random 
class bingocage:
    def __init__(self, items):
        self._items = list(items)
        random.shuffle(self._items)
    def pick(self):
        return self._items[0]
    def __call__(self):
        return self.pick()
ccc = bingocage([1,2,3])
x = ccc()
print(x)


1
2


### python也支持函数式编程，虽然创始人的目标不在于此。
### 主要得益于operator 和functools等包的支持

In [8]:
# map和reduce
x = map(lambda x:x+1,[1,2,3,4])
print(list(x))

from functools import reduce 
y = reduce(lambda x,y:x+y,[1,2,3,4])
print(y)

[2, 3, 4, 5]
10


In [60]:
# __str__ 和__repr__
class D:
    def __str__(self):
        return "str"
    def __repr__(self):
        return "repr"
item = D()
print(item)

x = "%s" % item
print(x)
y = "%r" % item
print(y)
item

str
str
repr


repr

---
## 下面主要讲一下个别编程模式
#### 《 设 计 模 式： 可 复 用 面 向 对 象 软 件 的 基 础》这本书讲了23种设计模式，16个实现方法
#### 下面从策略模式开始，使用<cite title='在python中，函数是一等对象。编程语言学家把“一等对象”定义为满足下述条件的程序实体：①在运行时创建。②能赋值给变量或数据结构中的元素。③能作为参数传给函数。④能作为函数的返回结果'>一等函数</cite>简化了这种实现方式<br>这意味着，很多种情况下，在python中使用函数或者可调用对象实现功能更自然，不用完全模仿书中所讲的“策略”，“命令”等模式。

In [64]:
# 策略模式
# 定义一系列算法，把他们封装起来，可替换，让客户自己选择，可以替换选择的那个方法或者类就是策略。应用策略的那个类就是上下文。
from abc import ABC, abstractmethod 
from collections import namedtuple 
Customer = namedtuple('Customer', 'name fidelity') 


class LineItem: 
    def __init__(self, product, quantity, price): 
        self.product = product 
        self.quantity = quantity 
        self.price = price 
        
    def total(self): 
        return self.price * self.quantity 
    
    
class Order: # 上 下 文 
    
    def __init__(self, customer, cart, promotion = None): 
        self.customer = customer 
        self.cart = list(cart) 
        self.promotion = promotion 
        
    def total(self):
        # 这里没有看明白，为什么要判断有没有__total属性
        if not hasattr(self, '__total'): 
            self.__total = sum(item.total() for item in self.cart) 
        return self.__total 
    
    def due(self): 
        if self.promotion is None: 
            discount = 0 
        else: 
            discount = self.promotion.discount(self) 
        return self.total() - discount 
    
    def __str__(self): 
        fmt = '< Order total: {:.2f} due: {:.2f} >' 
        return fmt.format(self.total(), self.due()) 
    
    
class Promotion(ABC) : # 策 略： 抽 象 基 类 
    @abstractmethod 
    def discount(self, order): 
        """ 返 回 折 扣 金 额（ 正 值）""" 

        
class FidelityPromo(Promotion): # 第 一 个 具 体 策 略
    """ 为 积 分 为 1000 或 以 上 的 顾 客 提 供 5% 折 扣""" 
    def discount(self, order): 
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0 
    
    
class BulkItemPromo( Promotion): 
    # 第 二 个 具 体 策 略 """ 单 个 商 品 为 20 个 或 以 上 时 提 供 10% 折 扣""" 
    def discount(self, order): 
        discount = 0 
        for item in order.cart: 
            if item.quantity >= 20: 
                discount += item.total() * .1 
        return discount 

    
class LargeOrderPromo( Promotion): 
    # 第 三 个 具 体 策 略 """ 订 单 中 的 不 同 商 品 达 到 10 个 或 以 上 时 提 供 7% 折 扣""" 
    def discount(self, order): 
        distinct_items = {item.product for item in order.cart} 
        if len(distinct_items) >= 10: 
            return order.total() * .07 
        return 0

 
joe = Customer(' John Doe', 0)
ann = Customer(' Ann Smith', 1100)
cart = [LineItem(' banana', 4, .5), 
        LineItem(' apple', 10, 1.5),
        LineItem(' watermellon', 5, 5.0)]
x = Order(joe, cart, FidelityPromo())
print(x)
y = Order(ann, cart, FidelityPromo())
print(y)






< Order total: 42.00 due: 42.00 >
< Order total: 42.00 due: 39.90 >


In [62]:
# 同上，使用函数实现
from abc import ABC, abstractmethod 
from collections import namedtuple 
Customer = namedtuple('Customer', 'name fidelity') 


class LineItem: 
    def __init__(self, product, quantity, price): 
        self.product = product 
        self.quantity = quantity 
        self.price = price 
        
    def total(self): 
        return self.price * self.quantity 
    
    
class Order: # 上 下 文 
    def __init__(self, customer, cart, promotion = None): 
        self.customer = customer 
        self.cart = list(cart) 
        self.promotion = promotion 
        
    def total(self):
        # 这里没有看明白，为什么要判断有没有__total属性
        if not hasattr(self, '__total'): 
            self.__total = sum(item.total() for item in self.cart) 
        return self.__total 
    
    def due(self): 
        if self.promotion is None: 
            discount = 0 
        else: 
            discount = self.promotion(self) 
        return self.total() - discount 
    
    def __str__(self): 
        fmt = '< Order total: {:.2f} due: {:.2f} >' 
        return fmt.format(self.total(), self.due()) 
    
        
def fidelity_promo(order): # 第 一 个 具 体 策 略
    """ 为 积 分 为 1000 或 以 上 的 顾 客 提 供 5% 折 扣""" 
    return order.total() * .05 if order.customer.fidelity >= 1000 else 0 
    
    
def bulk_item_promo(order): 
    # 第 二 个 具 体 策 略 """ 单 个 商 品 为 20 个 或 以 上 时 提 供 10% 折 扣""" 
    discount = 0 
    for item in order.cart: 
        if item.quantity >= 20: 
            discount += item.total() * .1 
    return discount 

    
def large_order_promo(order): 
    # 第 三 个 具 体 策 略 """ 订 单 中 的 不 同 商 品 达 到 10 个 或 以 上 时 提 供 7% 折 扣""" 
    distinct_items = {item.product for item in order.cart} 
    if len(distinct_items) >= 10: 
        return order.total() * .07 
    return 0

 
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5), 
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
x = Order(joe, cart, fidelity_promo)
print(x)
y = Order(ann, cart, fidelity_promo)
print(y)

# 这两个方法相比较。有这样一个结论,如果策略只是处理上下文,只有一个方法的话,那么就不要写成一个类,太浪费.写成一个普通的方法就好,轻便,而且也能共享.




< Order total: 42.00 due: 42.00 >
< Order total: 42.00 due: 39.90 >


In [65]:
# 再一次改进上面的脚本.---增加自动选择最大优惠的方法
promos = [fidelity_promo, bulk_item_promo, large_order_promo]


def best_promo(order):
    '''选择可用的最佳折扣'''
    return max(promo(order) for promo in promos)

# 这样可以用，但是如果添加了新的策略，必须要在新增函数的同时修改promos列表，容易出错，所以必须要改进,下面就来改进这个问题
# 利用globals,它的作用是返回一个字典，表示当前的全局符号表。只是在原始定义的脚本里，引用的不包含。
promos = [globals()[name] for name in globals() 
         if name.endswith('_promo')
         and name != 'best_promo']

# 另外一种收集所有策略的方法是把所有策略保存到一个单独的模块，这样就不用顾忌其他的函数比如这个脚本里的best_promo的干扰了

### 命令模式
可以通过吧函数作为参数而简化
没有深入的讲，
如果想要精通各种模式，那么请看下面这本书<br>
**《设 计 模 式： 可 复 用 面 向 对 象 软 件 的 基 础》**
